In [1]:
import os
import os.path as osp
import numpy as np
import pandas as pd
import argparse

from typing import Tuple, List, Dict, Any

from ukge.datasets import KGTripleDataset
from ukge.models import TransE, DistMult, ComplEx, RotatE
from ukge.losses import compute_det_transe_distmult_loss, compute_det_complex_loss, compute_det_rotate_loss
from ukge.metrics import Evaluator


from typing import Tuple, List, Dict, Any
import torch.nn.functional as F
from torch import Tensor
from torch.nn import Embedding

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

model_map = {
    'transe': TransE,
    'distmult': DistMult,
    'complex': ComplEx,
    'rotate': RotatE
}

model = 'distmult'
dataset = 'cn15k'
confidence_score_function = 'logi'
hidden_dim = 128
num_neg_per_positive = 10
batch_size = 1024
lr = 0.01
weight_decay = 0.005

model_checkpoint = '/home/mou/Projects/UKGE-FL/results/unc_cn15k_distmult_confi_logi/lr_0.01_wd_0.0_dim_128/best_model_ndcg_lin_topk.pth'

In [2]:
train_dataset = KGTripleDataset(dataset=dataset, split='train', num_neg_per_positive=num_neg_per_positive)
val_dataset = KGTripleDataset(dataset=dataset, split='val')
test_dataset = KGTripleDataset(dataset=dataset, split='test')
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = DistMult(num_nodes=train_dataset.num_cons(), num_relations=train_dataset.num_rels(), hidden_channels=hidden_dim, confidence_score_function=confidence_score_function)
checkpoint = torch.load(model_checkpoint)
print('best epoch:', checkpoint['best_ndcg_lin_topk_epoch'], 'best ndcg:', checkpoint['best_ndcg_lin_topk'])
model.load_state_dict(checkpoint['state_dict'])
model = model.to(device)

TypeError: KGTripleDataset.__init__() got an unexpected keyword argument 'deterministic'

In [ ]:
for name, param in model.named_parameters():
    print(name, param)

node_emb.weight Parameter containing:
tensor([[ 0.0037, -0.0051,  0.0046,  ...,  0.0234,  0.0050,  0.0148],
        [ 0.1428, -0.2090, -0.8836,  ...,  0.2341, -2.0511,  0.2387],
        [-0.4002, -0.2749,  0.3445,  ...,  0.1498, -0.2126, -0.3360],
        ...,
        [ 0.0166,  0.0151,  0.0171,  ..., -0.0195, -0.0048, -0.0044],
        [-0.0075,  0.0192, -0.0107,  ...,  0.0038, -0.0233,  0.0141],
        [-0.0061,  0.0135,  0.0176,  ..., -0.0052,  0.0110,  0.0131]],
       device='cuda:0', requires_grad=True)
rel_emb.weight Parameter containing:
tensor([[ 7.3744e-01,  1.2465e+00, -5.7172e-01,  ..., -1.5302e+00,
          4.8558e-01,  1.0224e-01],
        [ 4.4418e-02,  2.2577e+00,  8.9711e-01,  ...,  1.4587e-01,
          6.9473e-01,  8.9794e-01],
        [ 1.9164e+00,  1.7802e+00,  8.2396e-01,  ..., -1.0142e+00,
          9.0415e-01,  1.3774e+00],
        ...,
        [-2.5363e-02, -8.5877e-02,  8.3884e-03,  ...,  1.7362e-01,
          9.2349e-02,  3.3527e-03],
        [ 4.6411e-01, 

In [ ]:
h, r, t = 3442, 0, 594

plau_score = model.get_plausibility_score(torch.tensor([h]).to(device), torch.tensor([r]).to(device), torch.tensor([t]).to(device))
print('plausibility score:', plau_score)
fc_score = model.fc(plau_score.view(-1, 1)) 
print('fc score:', fc_score)
confi_score = F.sigmoid(fc_score)
print('confidence score:', confi_score)

plausibility score: tensor([-0.1009], device='cuda:0', grad_fn=<SumBackward1>)
fc score: tensor([[-39.9633]], device='cuda:0', grad_fn=<MmBackward0>)
confidence score: tensor([[4.4073e-18]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [5]:
bias = [False, True]
lrs = [0.01, 0.001, 0.0001]
hidden_dims = [128, 256, 512]
models = ['transe', 'distmult', 'complex', 'rotate']
datasets = ['cn15k', 'nl27k', 'ppi5k']
confi_score_functions = ['logi', 'rect']
fc_layers = ['l1', 'l3', 'none']


for b in bias:
    for lr in lrs:
        for hidden_dim in hidden_dims:
            for model in models:
                for dataset in datasets:
                    for confi in confi_score_functions:
                        for fc in fc_layers:
                            if b:
                                print(f"nohup python -u src/ukge/train_unc_baseline.py --model {model} --dataset {dataset} --confidence_score_function {confi} --fc_layers {fc} --bias --hidden_dim {hidden_dim} --lr {lr} --topk &> logs/unc_{model}_{dataset}_confi_{confi}_fc_{fc}_bias_{b}_lr_{lr}_dim_{hidden_dim}.log & ")
                            else: 
                                print(f"nohup python -u src/ukge/train_unc_baseline.py --model {model} --dataset {dataset} --confidence_score_function {confi} --fc_layers {fc} --hidden_dim {hidden_dim} --lr {lr} --topk &> logs/unc_{model}_{dataset}_confi_{confi}_fc_{fc}_bias_{b}_lr_{lr}_dim_{hidden_dim}.log & ")

nohup python -u src/ukge/train_unc_baseline.py --model transe --dataset cn15k --confidence_score_function logi --fc_layers l1 --hidden_dim 128 --lr 0.01 --topk &> logs/unc_transe_cn15k_confi_logi_fc_l1_bias_False_lr_0.01_dim_128.log & 
nohup python -u src/ukge/train_unc_baseline.py --model transe --dataset cn15k --confidence_score_function logi --fc_layers l3 --hidden_dim 128 --lr 0.01 --topk &> logs/unc_transe_cn15k_confi_logi_fc_l3_bias_False_lr_0.01_dim_128.log & 
nohup python -u src/ukge/train_unc_baseline.py --model transe --dataset cn15k --confidence_score_function logi --fc_layers none --hidden_dim 128 --lr 0.01 --topk &> logs/unc_transe_cn15k_confi_logi_fc_none_bias_False_lr_0.01_dim_128.log & 
nohup python -u src/ukge/train_unc_baseline.py --model transe --dataset cn15k --confidence_score_function rect --fc_layers l1 --hidden_dim 128 --lr 0.01 --topk &> logs/unc_transe_cn15k_confi_rect_fc_l1_bias_False_lr_0.01_dim_128.log & 
nohup python -u src/ukge/train_unc_baseline.py --mod